In [1]:
"""Collects data for file_properties_df.

file_properties_df holds metadata about data in our dataset, 
details of metadata can be seen at src/nna/tests/mock_data.py::mock_file_properties_df


"""


'Collects data for file_properties_df.\n\nfile_properties_df holds metadata about data in our dataset, \ndetails of metadata can be seen at src/nna/tests/mock_data.py::mock_file_properties_df\n\n\n'

In [2]:

from pathlib import Path
import pandas as pd
import datetime

from collections import Counter

from nna.fileUtils import list_files,getLength,read_file_properties_v2


In [3]:
# External Programs
# ffprobe version >= 4.3.1  
ffprobe_path = '/scratch/enis/conda/envs/speechEnv/bin/ffprobe'

In [6]:
# PARAMETERS for new DATABASE

# increase version number accordinly 
previous_database_ver_str = 'V5'
new_database_ver_str = 'V6'

# where to save txt file storing length info
# old_data_folder="/home/enis/projects/nna/data/"
data_folder = '/scratch/enis/data/nna/database/'
#/scratch/enis/data/nna/database

# path to search for audio files
# where
# ignore_folders=['/tank/data/nna/real/stinchcomb/']

search_path="/tank/data/nna/real/"

ignore_folders=["/tank/data/nna/real/stinchcomb/dups/","/tank/data/nna/real/stinchcomb/excerpts/"]
# search_path="/tank/data/nna/real/stinchcomb/"



In [7]:
# # PARAMETERS for external or small inferences

# # PARAMETERS  

# # increase version number accordinly 
# previous_database_ver_str = ''
# new_database_ver_str = 'V1'

# # where to save txt file storing length info
# # old_data_folder="/home/enis/projects/nna/data/"
# # data_folder = '/scratch/enis/data/nna/database/'
# data_folder = '/scratch/enis/data/nna/collar_database/'

# #/scratch/enis/data/nna/database

# # path to search for audio files
# # where
# # ignore_folders=['/tank/data/nna/real/stinchcomb/']

# # search_path="/tank/data/nna/real/"
# search_path = '/tank/data/nna/audio_collars/'

# ignore_folders=["/tank/data/nna/real/stinchcomb/dups/","/tank/data/nna/real/stinchcomb/excerpts/"]
# # search_path="/tank/data/nna/real/stinchcomb/"



In [8]:
# create Relative Path names

# if we already have a list of files we can load them 
# files_list_path=data_folder+"stinchcomb_files_pathV1.txt"
files_list_path=data_folder+ f"allFields_path{new_database_ver_str}.txt"

# if we calculated audio lengths and saved them into text file, 
# we can load them
fileswlen_path = data_folder+ f"allFields_wlen_f{new_database_ver_str}.txt"
filesWError_out = data_folder+f"allFields_wERROR_f{new_database_ver_str}.txt"

# do NOT add pkl extension at the end
pkl_file_name=f"allFields_data{new_database_ver_str}"


# this is the current info we have so we can check if we already processed a file before
current_pkl_file = data_folder + f"allFields_data{previous_database_ver_str}.pkl"

In [9]:
%%time
# Find files
# in given search path ignoring given directories
if not Path(fileswlen_path).exists():
    files_path_list=list_files(search_path,ignore_folders)
else:
    with open(files_list_path,"r") as f:
        lines=f.readlines()
        files_path_list=[line.strip() for line in lines]


CPU times: user 1.99 s, sys: 840 ms, total: 2.83 s
Wall time: 14.7 s


In [10]:
print('example file',files_path_list[0])


example file /tank/data/nna/real/anwr/31/2019/S4A10297_20190504_000000.flac


In [11]:
Path(files_path_list[0]).is_dir()

False

In [12]:
# count file extension and filter if required

files_suffixes=[]
files_path_list_filtered=[]
for m in files_path_list:
    m=Path(m)
    if m.is_dir():
        continue
    mSuffix = m.suffix.lower()
    
    files_suffixes.append(mSuffix)
    m_str=str(m).lower()
    if "~" in m_str:
        continue
    if "filepart" in m_str:
        continue
    if ".txt" in m_str:
        continue
#     if mSuffix!=".flac" and mSuffix!=".aac" and mSuffix!=".mp3":
#         print(m)
#         break
    files_path_list_filtered.append((m))
        
print(Counter(files_suffixes))
files_path_list = files_path_list_filtered[:]

Counter({'.flac': 165976, '.aac': 9101, '.mp3': 388, '.flac~': 1, '.txt': 1})


In [36]:
#V4: Counter({'.flac': 112053, '.aac': 9101, '.mp3': 388, '.flac~': 1})
#V5: Counter({'.flac': 151527, '.aac': 9101, '.mp3': 388, '.flac~': 1})
#V6: Counter({'.flac': 165976, '.aac': 9101, '.mp3': 388, '.flac~': 1, '.txt': 1})

'/scratch/enis/data/nna/collar_database/allFields_data.pkl'

In [13]:
# 
# Load previous data
if Path(current_pkl_file).exists() :

    current_file_properties_df=pd.read_pickle(str(current_pkl_file))

    # remove files we already know about
    currentFileSet = set(current_file_properties_df.index)
    foundFileSet = set(files_path_list)
    foundFileSet = foundFileSet.difference(currentFileSet)
    New_files_path_list = list(foundFileSet)
    a_str="new",len(New_files_path_list),"previously",len(currentFileSet),"total",len(files_path_list)
    print(a_str)
else:
    print('does NOT exists',current_pkl_file)
    New_files_path_list = files_path_list[:]


('new', 14450, 'previously', 161015, 'total', 175465)


In [48]:
# New_files_path_list

In [14]:
# Load or calculate Audio length
import subprocess

filesWError = []

# learn length of each audio and store in a text file, 
# if file already exists, it tries to get data from there
if not Path(fileswlen_path).exists():
    length_dict={}
    for f in New_files_path_list:
#         length=float(getLength(f))
##################
        input_video = f
        
        cmd=[]
        cmd.extend( [ffprobe_path, '-i', '{}'.format(input_video), '-show_entries' ,'format=duration', '-v', 'quiet' ])
        result = subprocess.Popen(cmd, stdout=subprocess.PIPE,stderr=subprocess.PIPE,)
        output = result.communicate(b'\n')
        output = [i.decode('ascii') for i in output]
        if output[0]=="":
            length = -1
            print("ERROR file is too short {}".format(input_video))
            print("command run with ERROR: {}".format(cmd))
            filesWError.append(input_video)
        else:
            length = output[0].split("\n")[1].split("=")[1]
###############
        length_dict[f]=length

    length_list=list(length_dict.items())
    Path(fileswlen_path).parent.mkdir(parents=True,exist_ok=True)
    with open(fileswlen_path,"w") as f:
        for line in length_list:
            join_lines = [str(i) for i in line]
            f.write(",".join(join_lines)+"\n")

with open(fileswlen_path,"r") as f:
    lines=f.readlines()
    fileswlen=[line.strip().split(",") for line in lines]

ERROR file is too short /tank/data/nna/real/dalton/04/2019/S4A10281_A_Summary.aac
command run with ERROR: ['/scratch/enis/conda/envs/speechEnv/bin/ffprobe', '-i', '/tank/data/nna/real/dalton/04/2019/S4A10281_A_Summary.aac', '-show_entries', 'format=duration', '-v', 'quiet']


In [16]:
filesWError

[PosixPath('/tank/data/nna/real/dalton/04/2019/S4A10281_A_Summary.aac')]

In [17]:
# print and save files with errors

print(len(filesWError))
with open(filesWError_out,"w") as f:
    for line in length_list:
        join_lines = [str(i) for i in line]
        f.write(",".join(join_lines)+"\n")

1


In [18]:
# turn results into a dict
fileswlen=dict([(i[0],float(i[1])) for i in fileswlen])

In [19]:
# file_properties
file_properties,exceptions = read_file_properties_v2(New_files_path_list,debug=0)


In [24]:
len(exceptions),len(New_files_path_list)

(1, 14450)

In [25]:
New_files_path_list[0]

PosixPath('/tank/data/nna/real/prudhoe/12/2021/S4A10265_20210706_060002.flac')

In [26]:
def str2timestamp(fileinfo_dict):
    # x=file_properties[file]
    #         print(x)
    hour_min_sec = fileinfo_dict["hour_min_sec"]
    hour = int(hour_min_sec[:2])
    minute = int(hour_min_sec[2:4])
    second = int(hour_min_sec[4:6])
    year = int(fileinfo_dict["year"])

    timestamp = datetime.datetime(year,
                                  int(fileinfo_dict["month"]),
                                  int(fileinfo_dict["day"]),
                                  hour=hour,
                                  minute=minute,
                                  second=second,
                                  microsecond=0)
    fileinfo_dict["timestamp"] = timestamp
    return fileinfo_dict

In [28]:
file_properties[Path('/tank/data/nna/real/prudhoe/12/2021/S4A10265_20210706_060002.flac')]

{'site_id': '12',
 'locationId': '12',
 'site_name': '',
 'recorderId': 'S4A10265',
 'hour_min_sec': '060002',
 'year': '2021',
 'month': '07',
 'day': '06',
 'region': 'prudhoe',
 'timestamp': datetime.datetime(2021, 7, 6, 6, 0, 2)}

In [29]:
# # hard code file_path
# file_properties = {}

# for apath in New_files_path_list:
#     startDateTime=apath.stem.split('_')
#     date = startDateTime[0]
#     year, month, day = date[0:4], date[4:6], date[6:8]
#     hour_min_sec = startDateTime[1]

#     # location_id = '149'
#     # if '721' in str(apath):
#     #     region = '721'
#     # elif '781' in str(apath):
#     #     region = '781'
    
#     site_name=''
#     recorderId=''
#     file_properties[apath] = str2timestamp({
#                     "site_id": location_id,
#                     "locationId": location_id,
#                     "site_name": site_name,
#                     "recorderId": recorderId,
#                     "hour_min_sec": hour_min_sec,
#                     "year": year,
#                     "month": month,
#                     "day": day,
#                     "region": region
#                 })


In [30]:
# file_properties,exceptions = read_file_properties_v2(New_files_path_list,debug=0)
for f,lengthSeconds in fileswlen.items():
    if file_properties.get(Path(f)) is not None:
        file_properties[Path(f)]["durationSec"] = lengthSeconds
        file_properties[Path(f)]["timestampEnd"] = file_properties[Path(f)]["timestamp"] + datetime.timedelta(seconds=lengthSeconds)
file_properties_df=pd.DataFrame(file_properties).T


In [31]:
file_properties_df = file_properties_df[file_properties_df['durationSec']>0]

In [34]:
# merge with previous file properties
merged_file_properties_df = pd.concat([file_properties_df,current_file_properties_df])


In [35]:
file_properties_df.to_pickle(data_folder+pkl_file_name+".pkl")

### Keep only prudhoe and anwr filter others
##### since they are only ones with images

In [ ]:
# file_properties_df = pd.read_pickle(str(file_properties_df_FilePath))


In [143]:
len(merged_file_properties_df)

121541

In [144]:
def mask2(df, key, value):
    return df[df[key] == value]
pd.DataFrame.mask2 = mask2

In [145]:
prudhoe = merged_file_properties_df.mask2("region",'prudhoe')
anwr = merged_file_properties_df.mask2("region",'anwr')

In [146]:
len(prudhoe),len(anwr)

15778

In [ ]:
prudhoeAndAnwr4photoExp = pd.concat([prudhoe,anwr])


In [154]:
len(prudhoeAndAnwr4photoExp)

30466

In [152]:
prudhoeAndAnwr4photoExp.to_pickle(data_folder+"prudhoeAndAnwr4photoExp_dataV1"+".pkl")

In [155]:
(data_folder+"prudhoeAndAnwr4photoExp_dataV1"+".pkl")

'/home/enis/projects/nna/data/prudhoeAndAnwr4photoExp_dataV1.pkl'

In [12]:
ex = '/home/enis/tmp/dempster-11_1,0.pkl'
import numpy as np

In [13]:

def load_clipping_2dict(
    region_location_name,
    clipping_results_path,
    threshold: float = 1.0,
    gathered_results=None,
):
    """Load clipping results into a dictionary.
file_properties_df, region_location_name,
                                    clipping_results_path
    """
    clipping_results_path = Path(clipping_results_path)
    if not gathered_results:
        gathered_results = {}
    clipping_threshold_str = str(threshold)
    clipping_threshold_str = clipping_threshold_str.replace(".", ",")
    file_name = (clipping_results_path /
                 (region_location_name + f"_{clipping_threshold_str}.pkl"))
    results_dict = np.load(file_name, allow_pickle=True)
    results_dict = results_dict[()]
    gathered_results.update(results_dict)  #type: ignore
    return gathered_results


In [14]:
results_dict = np.load(ex, allow_pickle=True)
results_dict = results_dict[()]

In [15]:
asd=(list(results_dict.keys()))

In [16]:
len(asd)

1384

In [6]:
asd.append('/tank/data/nna/real/dempster/14/2020/S4A10424_20200812_160000.flac')

In [8]:
from nna import clippingutils
all_results_dict, files_w_errors = clippingutils.run_task_save(
        asd,
        'dempster-11',
        '/home/enis/tmp',
        1.0,
    )


Clipping file for dempster-11_1,0.pkl exists at /home/enis/tmp. Checking existing results.
1 number of files missing results, calculating only those.


In [1]:
# TEMP #

In [4]:
import glob,os

In [5]:
def find_files_oswalk():
    for root, dirnames, filenames in os.walk('/mnt/sdi/Spring/'):
        for filename in filenames:
            yield os.path.join(root, filename)

In [6]:
all_files = []
for f in find_files_oswalk():
    all_files.append(f)

In [7]:
last_set=set()
counter=0
for f in all_files:
    last=f.split('.')[-1]
    last_set.add(last)
    if last=='wav':
        counter+=1


In [8]:
last_set,counter

({'SM4S', 'txt', 'wav'}, 7902)

In [20]:
15468/

SyntaxError: invalid syntax (<ipython-input-20-145b30d5bde8>, line 1)

In [ ]:
516x30